In [41]:
from typing_extensions import Annotated, TypedDict
from langchain_groq import ChatGroq, chat_models
from langgraph.graph import StateGraph, START, END
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
import os
load_dotenv()

True

In [42]:
template = """
You are a technical documentation assistant.

You MUST output Markdown that conforms exactly to the following rules.
This output will be parsed by a strict Markdown-to-Notion converter.

ALLOWED BLOCK SYNTAX:
- Headings: #, ##, ###
- Paragraphs (plain text)
- Bulleted lists using "- "
- Numbered lists using "1. "
- Blockquotes using "> "
- Code blocks using triple backticks: ```language
- Horizontal rules using "---"
- Tables using "|" syntax with PLAIN TEXT CELLS ONLY

ALLOWED INLINE SYNTAX:
- **bold**
- *italic*
- `inline code`
- [text](url)

STRICTLY FORBIDDEN:
- LaTeX or math notation (\( \), \[ \])
- HTML
- Unicode math symbols
- Nested Markdown
- Bold or italic inside tables
- Heading levels deeper than ###

If content cannot be expressed using the rules above,
rewrite it as plain text instead.

Follow these rules strictly.
Message = {message}
"""

prompt = ChatPromptTemplate.from_template(template=template)

In [43]:
class State(TypedDict):
    message: str

In [44]:
GROQ__API_KEY = os.getenv("GROQ_API_KEY")

In [45]:
def chat(state: State) -> State:
    model = ChatGroq(model="openai/gpt-oss-120b", api_key=GROQ__API_KEY)
    prompt_value = prompt.invoke({"message": state["message"]})
    return {"message": model.invoke(prompt_value).content}

In [46]:
graph = StateGraph(State)
graph.add_node("chat_node", chat)
graph.add_edge(START, "chat_node")
graph.add_edge("chat_node", END)

In [47]:
graph = graph.compile()

In [48]:
message = "What is an LLM"

In [49]:
op = graph.invoke({"message": message})

In [50]:
op

{'message': '# Large Language Model (LLM)\n\nA **large language model** (LLM) is a type of artificial intelligence that has been trained on massive amounts of text data. It learns statistical patterns in language and can generate human‑like text, answer questions, translate languages, and perform many other language‑related tasks.\n\n## Core Concepts\n\n- **Training data** – billions of words from books, articles, websites, and other sources.  \n- **Neural network architecture** – usually a transformer model that processes text in parallel.  \n- **Parameters** – the adjustable weights of the network; modern LLMs have from hundreds of millions to trillions of parameters.  \n- **Self‑supervised learning** – the model predicts missing words or next sentences without explicit labels.\n\n## How It Works\n\n1. **Tokenization** – input text is split into smaller units called tokens.  \n2. **Embedding** – each token is converted into a numeric vector that captures its meaning.  \n3. **Attentio